# Minimization of Multivariate Functoins

We now move into minimizing objectives that are multivariate functions.  They still return a single quantity that we wish to optimize, so they are scalar functions, but we will now move into the case of optimizing that objective function by iteratively varying ***more than one*** function argument. We encounter this type of problem all the time!

Let's start with a very basic example: we have a function that describes the value of z for values of x and y:

$$z(x,y) = (x - 10)^2 + (y + 5)^2$$

By inspection, we know that this function has a minimum value of z = 0 at x = 10, y = -5, but if we can, it's a good idea to start with a visualization of the problem.  This is a 3D problem, so we are still able to visualize it reasonably well. Once we hit 4D, all bets are off!  

You can look up 3D plotting in matplotlib; I found the link below to be helpful. 

https://jakevdp.github.io/PythonDataScienceHandbook/04.12-three-dimensional-plotting.html

In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

## Visualizing a multivariate function with a surface plot

In [ ]:
# This plot graphs the above function z in a 3d surface plot

z    = lambda x,y: (x-10)**2 + (y+5)**2
x    = np.linspace(-50, 50, 100)
y    = np.linspace(-50, 50, 100)
X, Y = np.meshgrid(x, y) #we're making a surface plot, so we create a grid of (x,y) pairs
Z    = z(X,Y)  #generate the Z data on the meshgrid (X,Y) by evaluating f at each XY pair.

#Create the figure and axis
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
#Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap = 'jet')
#Set properties
plt.xlim(-50, 50)
plt.xticks(np.arange(-50, 50.1, 20))
plt.ylim(-50, 50)
plt.yticks(np.arange(-50, 50.1, 20))
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Z values vs. X and Y')
plt.show()

## Analogous syntax for contour plots and filled contour plots

In [ ]:
#Plot as contours
plt.figure(2, figsize = (7, 5))
plt.title('A contour plot of Z')
plt.contour(X, Y, Z, levels = 50)
plt.xlim(-50, 50)
plt.xticks(np.arange(-50, 50.1, 20))
plt.ylim(-50, 50)
plt.yticks(np.arange(-50, 50.1, 20))
plt.xlabel('X', fontsize = 12)
plt.ylabel('Y', fontsize = 12)
plt.colorbar()
plt.show()

In [ ]:
#Plot as filled contours
plt.figure(2, figsize = (7, 5))
plt.title('A filled contour plot of Z')
plt.contourf(X, Y, Z, levels = 50, cmap = 'jet')
plt.xlim(-50, 50)
plt.xticks(np.arange(-50, 50.1, 20))
plt.ylim(-50, 50)
plt.yticks(np.arange(-50, 50.1, 20))
plt.xlabel('X', fontsize = 12)
plt.ylabel('Y', fontsize = 12)
plt.colorbar()
plt.show()

## Passing array arguments to the function

This is a multivariate function, z(x,y), that returns a single output, z. We can minimize z by using any of the `opt.minimize()` routines or any of the global optimzation routines we tested on univariate functions. We just make a simple extention to our original syntax. I know that above, we've wrote this out as z(x,y), which is conceptually true, but there is a catch when working with `opt.minimize()`:

It will only vary the value of the first argument when seeking an optimum, and it will treat any additional arguments we pass to the function as fixed parameters. So, if I were to give z(x,y) to an optimizer, it would vary the value of x while holding y fixed at the initial value. Instead, I have to create function that accepts an ***array*** as its first argument. That first argument should be a collection - usually a list, a numpy array, or a tuple - of variables that I want to adjust until I find the minimum value of my function.  So, instead of working with:

```python
def z(x,y):
    return (x - 10)**2 + (y + 5)**2
```
        
or it's analogous lambda function

```python
z = lambda x,y: (x - 10)**2 + (y + 5)**2
```

We want to write this particular objective function such that its first argument is a collection of all of the variables we want to minimize with respect to:

```python
def z(var):
    return (var[0] - 10)**2 + (var[1] + 5)**2
```

or, it's analgous lambda function:

```python
z = lambda var: (var[0] - 10)**2 + (var[1] + 5)**2
```

### Improving human readability
    
Often, to improve readability in a complex function, I will use a long form function definition and redefine elements in "var" using the labels that are specific to the way we express our function on paper:

```python
def z(var):
    x = var[0]
    y = var[1]
    return (x - 10)**2 + (y + 5)**2
```

Note that Python will also allow you to unpack elements in an array as a set of comma separated values, which is a bit more concise (This is equivalent in practice to the above).

```python
def z(var):
    x, y = var
    return (x - 10)**2 + (y + 5)**2
```

Any of the above options will work. Now that we've created an objective function that takes a single array argument, we can pass this objective function as an argument to `opt.minimize()`.  The only other catch is that we need an initial guess for the (x,y) pair, and it should be a collection of the same size and type that we used for our function argument - in fact, our initial guess is what sets the data type for var.  So, my full optimization script would look something like this. Just so that we can make the optimizer work for it, let's start at an initial guess of [10, 50] for x and y, which we'll pass as `var0 = [10, 50]`. As we saw in past exercises, `opt.minimize()` will output a solution structure, and I can access individual elements using a dot operator.

In [ ]:
def z(var):
    x, y = var
    return (x - 10)**2 + (y + 5)**2
var0 = [10, 50]  #Initial guess for x and y as a list; I could also use an array or tuple.
opt.minimize(z, var0)

In [ ]:
opt.minimize(z, var0, bounds = [(1, 10), (20, 50)])

### Passing extra arguments or parameters

Finally, you often will encounter an optimization problem wherein you need to minimize an objective function by varying a set of parameters, but you also need to pass additional information to the function.  As an example, let's use the following function:

$$q(x,y,a,b) = ax^2 + by^2 + x - y$$

In a lot of languages, you'd probably use anonymous functions to handle this; that will work in Python as well.  That said, Python also gives you the convenient option of passing extra arguments using the `args` keyword.  ***This works for most numerical methods from Scipy***.

In [ ]:
def q(var, a, b):
    x = var[0]
    y = var[1]
    return a*x**2 + b*y**2 + x - y

In [ ]:
opt.minimize(q, var0, args = (1, 3))

## Least Squares

The basic problem should be familiar to everyone. You have a set of data, and you want to create a model that describes data well enough that it is reasonably predictive. 

### Enzyme Kinetics

In [ ]:
CS = np.array([0, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 2, 5]) #mmol/L
rate = np.array([0, 0.017537467, 0.030941975, 0.080327165, 0.1643835, 0.26569368, 0.745442547, 1.295792328, 2.419014706, 4.0402125, 5.534947297, 5.127217742, 7.074911496]) #mmol/L/min

## Graphing the data
plt.figure(1, figsize = (5, 5))
plt.scatter(CS, rate, marker = 'o', color = 'none', edgecolor = 'black')
plt.xlim(0, 6)
plt.ylim(0, 8)
plt.xlabel('Substrate Concentration (mmol/L)', fontsize = 12)
plt.ylabel('Rate (mmol/L/min)', fontsize = 12)
plt.show()

### Minimizing Least Squares for Michaelis-Menten Kinetics

The "saturation kinetics" that we observe in this system suggests a Michaelis-Menten mechanisms, which we model as:

$$rate = \frac{V_{max}C_S}{K_m + C_S}$$

Next, we need to build an objective function that calculates the residual sum of squares between the model prediction and the experimental measurment for our experimental set of substrate concentrations. In this case, our objective function will be a ***multivariate scalar function***. It will accept two parameters that we want to find optimum values for -- $K_m$ and $V_{max}$ -- and it will return a single value that we wish to minimize, the residual sum of squares. 

In [ ]:
def obj(par, C, r): #obj(variable parameters, extra arg 1, extra arg 2)
    Vmax  = par[0]
    Km    = par[1]
    model = (Vmax*C)/(Km + C)
    SSE = sum((r - model)**2)
    return SSE

In [ ]:
par0 = np.array([1,1])
sol = opt.minimize(obj, par0, args = (CS, rate))
print(sol)

In [ ]:
opt.minimize(obj, [7,0.3], args = (CS, rate), bounds = [(6.8, 7.4), (0.1, 0.5)])